# Data Load

In [1]:
import pandas as pd

In [2]:
from modules.data_store import data_store

There is data stored
Last data fetch completed


In [3]:
data_store = data_store()

Load the saved pickle file


In [4]:
trial = data_store.return_last_trial()
df = data_store.return_dict_df()[trial]

## target 값 - input변수 norm변환후 계산

In [5]:
import modules.config as config
input_cols = config.trial_3_input_cols
input_cols

['DAT',
 'obs_hour',
 'change_내부온도관측치',
 '내부습도관측치',
 'co2관측치',
 'ec관측치',
 '시간당분무량',
 '일간누적분무량',
 'change_시간당백색광량',
 'change_일간누적백색광량',
 'change_시간당적색광량',
 'change_일간누적적색광량',
 'change_시간당청색광량',
 'change_일간누적청색광량',
 'change_시간당총광량',
 'change_일간누적총광량']

## EBM

In [6]:
base_df = df.copy()

In [7]:
train_df = base_df[base_df['train/test']=='CASE']
test_df = base_df[base_df['train/test']=='TEST']

In [8]:
from interpret.glassbox import ExplainableBoostingRegressor

X_train = train_df[input_cols]
y_train = train_df['target']

ebm_1 = ExplainableBoostingRegressor()
ebm_1.fit(X_train, y_train)

ExplainableBoostingRegressor(feature_names=['DAT', 'obs_hour', 'change_내부온도관측치',
                                            '내부습도관측치', 'co2관측치', 'ec관측치',
                                            '시간당분무량', '일간누적분무량',
                                            'change_시간당백색광량', 'change_일간누적백색광량',
                                            'change_시간당적색광량', 'change_일간누적적색광량',
                                            'change_시간당청색광량', 'change_일간누적청색광량',
                                            'change_시간당총광량', 'change_일간누적총광량',
                                            'DAT x change_시간당총광량',
                                            'DAT x 시간당분무량', '내부습도관측치 x 시간당분무량',
                                            '내부습도관측치 x 일간누적분무량',
                                            'DAT x change_일간누적총광량'...
                             feature_types=['continuous', 'continuous',
                                            'continuous', 'continuous',
                                

In [9]:
x_predict = ebm_1.predict(X_train)
X_train['predict_d1'] = pd.DataFrame(x_predict)

/Users/youngyong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:
X_train['predict_d1'].head(24).sum()

-0.845578258050341

In [11]:
def predict_rslt_weight(x):
    if x < 0 :
        return 0
    else : 
        return x

X_train['predict_d1'] = X_train['predict_d1'].apply(predict_rslt_weight)

/Users/youngyong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [12]:
X_train['predict_d1'].head(24).sum()

1.0645849670705119

In [13]:
ebm_2 = ExplainableBoostingRegressor()
ebm_2.fit(X_train, y_train)

ExplainableBoostingRegressor(feature_names=['DAT', 'obs_hour', 'change_내부온도관측치',
                                            '내부습도관측치', 'co2관측치', 'ec관측치',
                                            '시간당분무량', '일간누적분무량',
                                            'change_시간당백색광량', 'change_일간누적백색광량',
                                            'change_시간당적색광량', 'change_일간누적적색광량',
                                            'change_시간당청색광량', 'change_일간누적청색광량',
                                            'change_시간당총광량', 'change_일간누적총광량',
                                            'predict_d1',
                                            'ec관측치 x change_시간당적색광량',
                                            'ec관측치 x change_시간당총광량',
                                            'ec관측치 x change_일간누적백색광량',
                                            'co2관측...
                             feature_types=['continuous', 'continuous',
                                            'continuous', 'conti

In [15]:
x_predict = ebm_2.predict(X_train)
pd.DataFrame(x_predict)[:24].sum()

0    1.677147
dtype: float64

# test predict

In [16]:
X_test = test_df[input_cols]
y_test = test_df['target']


X_test['predict_d1'] = ebm_1.predict(X_test)
X_test['predict_d1'] = X_test['predict_d1'].apply(predict_rslt_weight)

/Users/youngyong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/youngyong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [17]:
test_df['predicted_weight_g'] = ebm_2.predict(X_test)

/Users/youngyong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [18]:
base_rslt_df = test_df[['DAT','file_num','predicted_weight_g']].groupby(['DAT','file_num']).sum().reset_index()
def predict_rslt_weight(x):
    if x < 0 :
        return 0
    else : 
        return x

base_rslt_df['predicted_weight_g'] = base_rslt_df['predicted_weight_g'].apply(predict_rslt_weight)
# base_rslt_df = base_rslt_df.set_index('DAT')
base_rslt_df

,DAT,file_num,predicted_weight_g
0,1,01,2.269210
1,1,02,1.111318
2,1,03,0.000000
3,1,04,0.943887
4,1,05,1.178283
...,...,...,...
135,28,01,97.016388
136,28,02,86.889096
137,28,03,38.505949
138,28,04,116.074823


In [19]:
file_num_list = set(base_rslt_df['file_num'])
test_target_list = [] 
file_path = './open/sample_submission/'
for file_num in file_num_list:
    test_target_file = file_path+'TEST_'+file_num+'.csv'
    print(test_target_file)
    test_target_list.append(test_target_file)
    base_rslt_df[base_rslt_df['file_num']==file_num][['DAT','predicted_weight_g']].to_csv(test_target_file,index=False)
    

./open/sample_submission/TEST_03.csv
./open/sample_submission/TEST_02.csv
./open/sample_submission/TEST_01.csv
./open/sample_submission/TEST_05.csv
./open/sample_submission/TEST_04.csv


- 성능이 8.5로 하락

---

In [22]:
target_df = base_df.groupby(['DAT','train/test','file_num','predicted_weight_g']).count().reset_index()[['DAT','train/test','file_num','predicted_weight_g']]
target_df.head()

,DAT,train/test,file_num,predicted_weight_g
0,1,CASE,01,0.167719
1,1,CASE,02,1.051178
2,1,CASE,03,0.685391
3,1,CASE,04,1.656917
4,1,CASE,05,4.223077


In [50]:
len(base_df)

22176

In [40]:
X_train['file_num'] = base_df['file_num']
X_train['target'] = x_predict
X_train_target = X_train.groupby(['DAT','file_num']).sum().reset_index()[['DAT','file_num','target']]
# X_train.head()

/Users/youngyong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/youngyong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [42]:
target_df.head()

,DAT,train/test,file_num,predicted_weight_g
0,1,CASE,01,0.167719
1,1,CASE,02,1.051178
2,1,CASE,03,0.685391
3,1,CASE,04,1.656917
4,1,CASE,05,4.223077


In [43]:
X_train_target.head()

,DAT,file_num,target
0,1,01,2.164235
1,1,02,0.789346
2,1,03,0.031673
3,1,04,-0.900578
4,1,05,4.535588


In [52]:
len(target_df[target_df['train/test'] == 'CASE'])*24

18816

In [64]:

tmp = pd.merge(target_df[target_df['train/test'] == 'CASE'], X_train_target,how='inner',on=['DAT','file_num'])
tmp['target'] = tmp['target'].apply(predict_rslt_weight)

tmp['diff'] = abs(tmp['predicted_weight_g'] - tmp['target'])
tmp.sort_values('diff',ascending=True).head(20)

,DAT,train/test,file_num,predicted_weight_g,target,diff
677,25,CASE,06,36.015762,36.018157,0.002395
136,5,CASE,25,2.098527,2.107579,0.009052
233,9,CASE,10,7.240668,7.229289,0.011380
75,3,CASE,20,2.773041,2.761225,0.011816
60,3,CASE,05,4.966084,4.950557,0.015527
39,2,CASE,12,3.228855,3.245588,0.016733
400,15,CASE,09,22.236957,22.213687,0.023270
279,10,CASE,28,8.493317,8.518425,0.025108
201,8,CASE,06,5.761957,5.730312,0.031645
441,16,CASE,22,7.251270,7.212237,0.039033


In [63]:
import pandas as pd
from sklearn.metrics import mean_squared_error

# df = pd.read_csv('testdata.csv', header=0, encoding='utf8')
rmse = mean_squared_error(tmp['target'], tmp['predicted_weight_g'],squared=False)
mse = mean_squared_error(tmp['target'], tmp['predicted_weight_g'])
rmse_2 = mse ** 0.5
print('mse',mse,'rmse',rmse)
print('rmse',rmse_2)

mse 18.60505570818259 rmse 4.313357822877971
rmse 4.313357822877971


In [41]:
X_train_target.describe()

,DAT,target
count,784.000000,784.000000
mean,14.500000,34.927331
std,8.082904,40.614713
min,1.000000,-3.771071
25%,7.750000,5.029470
50%,14.500000,17.517734
75%,21.250000,53.140611
max,28.000000,167.838807


In [33]:
target_df[target_df['DAT']>=25].describe()

,DAT,predicted_weight_g
count,132.000000,132.000000
mean,26.500000,76.877330
std,1.122293,49.379555
min,25.000000,0.000000
25%,25.750000,37.846409
50%,26.500000,78.564944
75%,27.250000,108.021004
max,28.000000,164.150300


In [34]:
# base_rslt_df.describe()
base_rslt_df[base_rslt_df['DAT']>=25].describe()

,DAT,predicted_weight_g
count,20.000000,20.000000
mean,26.500000,70.516051
std,1.147079,28.324240
min,25.000000,18.611551
25%,25.750000,57.391114
50%,26.500000,72.292160
75%,27.250000,89.071133
max,28.000000,116.074823


In [28]:
target_df[target_df['predicted_weight_g']==target_df['predicted_weight_g'].max()]

,DAT,train/test,file_num,predicted_weight_g
901,28,CASE,11,164.1503


In [30]:
base_rslt_df[base_rslt_df['predicted_weight_g']==base_rslt_df['predicted_weight_g'].max()]

,DAT,file_num,predicted_weight_g
138,28,04,116.074823


In [32]:
load_data(df,28,'TEST','04')

,DAT,predicted_weight_g,train/test,file_num,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,...,norm_일간누적분무량,norm_change_시간당백색광량,norm_change_일간누적백색광량,norm_change_시간당적색광량,norm_change_일간누적적색광량,norm_change_시간당청색광량,norm_change_일간누적청색광량,norm_change_시간당총광량,norm_change_일간누적총광량,total_sum
20136,28,0.0,TEST,04,00:00,27.461667,59.956667,395.083333,0.0,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,52.412883
20137,28,0.0,TEST,04,01:00,27.459322,60.151724,391.396552,0.0,126.00,...,0.002940,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,52.412883
20138,28,0.0,TEST,04,02:00,27.811666,60.061667,388.683333,0.0,0.00,...,0.002940,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,52.412883
20139,28,0.0,TEST,04,03:00,27.795000,59.988334,384.416667,0.0,0.00,...,0.002940,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,52.412883
20140,28,0.0,TEST,04,04:00,28.085000,60.091667,382.250000,0.0,126.00,...,0.005880,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,52.412883
20141,28,0.0,TEST,04,05:00,27.945000,60.370000,379.483333,0.0,0.00,...,0.005880,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,52.412883
20142,28,0.0,TEST,04,06:00,28.113333,62.710000,369.983333,0.0,252.00,...,0.011761,0.155167,0.009865,0.040000,0.001980,0.018167,0.000899,0.091008,0.006164,52.412883
20143,28,0.0,TEST,04,07:00,27.995000,67.518333,357.866667,0.0,126.00,...,0.014701,0.499000,0.041588,0.137833,0.008804,0.066667,0.004196,0.297127,0.026287,52.412883
20144,28,0.0,TEST,04,08:00,28.750000,68.336667,347.300000,0.0,252.00,...,0.020581,0.687333,0.085285,0.196000,0.018507,0.097667,0.009027,0.412328,0.054213,52.412883
20145,28,0.0,TEST,04,09:00,28.631667,69.893333,346.166667,0.0,126.00,...,0.023521,0.688333,0.129045,0.196667,0.028243,0.098333,0.013891,0.413158,0.082195,52.412883


In [29]:
def load_data(df,DAT,train_test,file_num):
    return df[(df['DAT']==DAT)&(df['train/test']==train_test)&(df['file_num']==file_num)]
load_data(df,28,'CASE','11')

,DAT,predicted_weight_g,train/test,file_num,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,...,norm_일간누적분무량,norm_change_시간당백색광량,norm_change_일간누적백색광량,norm_change_시간당적색광량,norm_change_일간누적적색광량,norm_change_시간당청색광량,norm_change_일간누적청색광량,norm_change_시간당총광량,norm_change_일간누적총광량,total_sum
18120,28,164.1503,CASE,11,00:00,20.525000,79.620000,445.300000,3.072686,769.00,...,0.017944,0.0,0.0,0.04,0.001980,0.026667,0.001319,0.017096,0.001158,83.654542
18121,28,164.1503,CASE,11,01:00,19.845000,80.045000,477.350000,3.092903,0.00,...,0.017944,0.0,0.0,0.00,0.001980,0.000000,0.001319,0.000000,0.001158,83.654542
18122,28,164.1503,CASE,11,02:00,19.716667,81.153333,483.483333,3.101475,1390.11,...,0.050381,0.0,0.0,0.00,0.001980,0.000000,0.001319,0.000000,0.001158,83.654542
18123,28,164.1503,CASE,11,03:00,19.077966,81.933899,479.084746,3.105443,0.00,...,0.050381,0.0,0.0,0.00,0.001980,0.000000,0.001319,0.000000,0.001158,83.654542
18124,28,164.1503,CASE,11,04:00,25.113333,79.971667,388.533333,3.080906,1377.25,...,0.082519,0.0,0.0,0.56,0.029703,0.373333,0.019786,0.239338,0.017367,83.654542
18125,28,164.1503,CASE,11,05:00,25.145000,79.605000,379.600000,3.090479,769.00,...,0.100463,0.0,0.0,0.59,0.058911,0.393333,0.039242,0.252160,0.034445,83.654542
18126,28,164.1503,CASE,11,06:00,25.532203,79.366101,390.305085,3.092742,1473.70,...,0.134851,0.0,0.0,0.58,0.087624,0.386667,0.058368,0.247886,0.051234,83.654542
18127,28,164.1503,CASE,11,07:00,25.605000,79.156667,389.933333,3.091639,769.00,...,0.152795,0.0,0.0,0.59,0.116832,0.393333,0.077824,0.252160,0.068312,83.654542
18128,28,164.1503,CASE,11,08:00,25.848333,78.925000,384.616667,3.103319,1934.06,...,0.197925,0.0,0.0,0.59,0.146040,0.393333,0.097279,0.252160,0.085390,83.654542
18129,28,164.1503,CASE,11,09:00,25.341667,78.786667,385.733333,3.118197,1165.06,...,0.225111,0.0,0.0,0.59,0.175248,0.393333,0.116735,0.252160,0.102468,83.654542
